[Index](Index.ipynb) - [Prev](Implementation_of_quantum_circuit.ipynb) - [Next](Rotation_gates.ipynb)

## Qiskit basics <a id ='basiccircuits'></a>

Qiskit general workflow goes as follow ([see tutorial](https://qiskit.org/documentation/intro_tutorial1.html)):

1. Build: Design a quantum circuit(s) that represents the problem you are considering.

2. Run/execute: Run the circuits on different backends

3. Analyze: Compute summary statistics and visualize the results of the experiments.


# Build a circuit

In [ ]:
# the python packages to import 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ
from qiskit.visualization import plot_bloch_multivector, plot_histogram
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, inset_axes
import numpy as np

In [ ]:
# defining the circuit of one qubit
q = QuantumRegister(1, 'q')
c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(q ,c)

# visualize the circuit
circuit.draw(output='mpl') 

Circuits are the fundamental unit of Qiskit. Most circuits are a combination of classical and quantum registers. The quantum registers are used to perform quantum operations on qubits. The classical registers are used to perform classical operations on the measurements obtained. The code above only creates a circuit and visualizes it, but we have not yet done anything with it. Lets now see how to execute it!

# Execute the circuit

Here we will use two backends:
- statevector_simulator: returns the quantum state as a complex vector of dimensions $2^N$ , where $N$ is the number of qubits.
- qasm_simulator: returns a bit string of the last result and the measurements probability distibution for 1024 executions

The state_simulator backend is usefull to visualize states and the effect of the gates. You should however not add any measurments in this case because when doing so, we lose the information about the state and collapse it to one of the basis states. The output state is always one of the basis states and this does not provide by itself with a lot of information about the circuit.

To get insight after a measurement, we need to perform the experiment several times. The interesting results are related with the probability distribution of outcomes. For this, we can use qasm_simulator backend and get back the number of times one results or the other was obtained. We further plot a histogram of the outcomes.


## statevector backend

In [ ]:
# define the backend
simulator = Aer.get_backend('statevector_simulator')

# defining the circuit
q = QuantumRegister(1, 'q')
c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(q,c)

# execute the circuit with the defined backend
job = execute(circuit, simulator).result()

# get statevector
state = job.get_statevector()
print("state of qubit = " + str(state))

# visualize state in bloch sphere
plot_bloch_multivector(state)



The circuit is executed and the results are stored in the object "job". From this object we can obtain the statevector. Unless specified differently, qubits are always initialize in state $|0\rangle$. 

## qasm_simulator backend

In [ ]:
# define simulator backend
simulator = Aer.get_backend('qasm_simulator')

#define circuit
q = QuantumRegister(1, 'q')
c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(q,c)

# Add a measurement to the circuit
circuit.measure(q,c)

# execute the circuit with the defined backend
job = execute(circuit, simulator).result()
counts = job.get_counts()
print ("total counts for |0> are:",counts)

# plot histogram
plot_histogram(counts)

In [ ]:
# visualize the circuit
circuit.draw(output='mpl') 

Notice that now we have added a measurement to the circuit. The measurement takes as arguments both the quantum and classical registers and uses the classical register to store the measurement outcomes. If you visualize the circuit, you will see an arrow pointing from the qubit to the classical bit used to store the extracted output.

Since we have done nothing to the qubit and the qubit is always initialize in state $|0\rangle$, the probability of measuring the qubit in state $|0\rangle$ is equal to 1.0.


> E1: Build a circuit with 4 qubits, add a measurment and visualize the outcome. 

## Initializing the qubit in different states
In some cases, we want to initialize our qubits in a different state, we can do this by using the following:

In [ ]:
# define the backend
simulator = Aer.get_backend('statevector_simulator')

#define circuit
q = QuantumRegister(1, 'q')
c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(q,c)

# Define initial_state alpha=0, beta=1
initial_state = [1,0]   
# Apply initialisation operation to the qubit 
circuit.initialize(initial_state, 0) # state and index of qubit to be initialized

# visualize initial_state in bloch sphere
plot_bloch_multivector(initial_state)

> E2: Initialize the qubit in state $\alpha = \frac{1}{\sqrt 2}$ and $\beta = \frac{1}{\sqrt 2}$ and visualize it in Bloch sphere. (hint: to get $\frac{1}{\sqrt 2}$ use the function np.sqrt())

> Q1: What do you expect the probability of measuring state $|0\rangle$ is?

Lets now add a measurment and execute the circuit:

In [ ]:
# define the backend
simulator = Aer.get_backend('qasm_simulator')

#define circuit
q = QuantumRegister(1, 'q')
c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(q,c)

# Define initial_state alpha=0, beta=1
initial_state = [0,1]   
# Apply initialisation operation to the qubit 
circuit.initialize(initial_state, 0) # state and index of qubit to be initialized

# Add a measurement to the circuit
circuit.measure(q,c)

# execute the circuit with
job = execute(circuit, simulator).result()

# plot histogram
counts = job.get_counts()
plot_histogram(counts)

#circuit.draw(output='mpl') 


> E3: Execute an experiment with the qubit initialized in state $\alpha = \frac{1}{\sqrt 2}$ and $\beta = \frac{1}{\sqrt 2}$ and plot the results. Is this what you expected?

> E4: lets know do the same with the qubit initialized in state $\alpha = \frac{1}{2}$ and $\beta = \frac{\sqrt 3}{2}$ and plot the results. Is this what you expected? Try also inverting the values of $\alpha$ and $\beta$

## Pauli gates 

Now that we know how to build circuits in qiskit and how to analyze some of the results (histogram, Bloch sphere and state vector), let us do something more interesting and start adding gates to the circuit. We will start by first using the statevector_simulator to visualize in the Bloch sphere the effect of the gates. We will execute the experiment but we wont measure it, as this collapses the state.

In [ ]:
# define the backend
simulator = Aer.get_backend('statevector_simulator')

#define circuit
q = QuantumRegister(1, 'q')
c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(q,c)

# Define initial_state alpha=1, beta=0 --> state |0>
initial_state = [1,0]   

# Apply initialisation operation to the qubit
circuit.initialize(initial_state, 0) 

# Apply X gate to qubit
circuit.x(0)

# ---- Add any extra gates here----

# Add a measurement to the circuit
#circuit.measure(q,c)

# execute the circuit 
job = execute(circuit, simulator).result()

# get statevector
state = job.get_statevector()
print("state of qubit = " + str(state))

# visualize state in bloch sphere
plot_bloch_multivector(state)

#visualize crcuit
#circuit.draw(output='mpl') 


> Q2: What is the effect of the X-gate?




> E4: Add a Y-gate instead of an X-gate. What changes in the qubit state? 

> E5: Uncomment the measurment of the circuit with a Y-gate. 

> Q3: What happens now with the qubit state? Is there any difference? Can you explain this?



> E6: Add a Z-gate instead of a Y-gate, initialize the qubit in state $\alpha = \frac{1}{\sqrt 2}$ and $\beta = \frac{1}{\sqrt 2}$ and comment again the measurment.

> Q4: What changed? 

> E7: Uncomment the measurment. What happens now?

> E8: Comment again the measurment and try to add two consecutive X-gates or two consecutive Y-gates. What do you observe?
